# Professor Adams - Financial Risk Management (FRE 6123) 
# Homework #3

### README

---
---
### Part 1 Set up Environment

In [2]:
import math
import pandas as pd
import numpy as np
import math
from dataclasses import dataclass
from typing import List, Tuple

---
---

### Part 2 Answer

#### Question 1 

In [3]:
# Credit risk metrics for a 4-year bullet loan

principal = 50_000_000.0  # USD
ytm = 0.03                # annual compounding
T = 4                     # years
recovery = 0.60
annual_pod = 0.0175       # per-year probability of default (conditional hazard)

# (a) Maximum exposure (just before maturity), bullet structure
ead_max = principal * ((1 + ytm) ** T)

# (b) Maximum Loss Given Default
lgd_max = (1 - recovery) * ead_max

# (c) Anticipated default rate in Year 4 (marginal PD in the 4th year)
survival_to_year4 = (1 - annual_pod) ** 3
marginal_pd_year4 = annual_pod * survival_to_year4

print("Max Expected Exposure (EAD_max) =", format(ead_max, ".6f"))
print("Max LGD =", format(lgd_max, ".6f"))
print("Survival to start of Year 4 =", format(survival_to_year4, ".6f"))
print("Anticipated default rate in Year 4 =", format(marginal_pd_year4, ".6f"))


Max Expected Exposure (EAD_max) = 56275440.500000
Max LGD = 22510176.200000
Survival to start of Year 4 = 0.948413
Anticipated default rate in Year 4 = 0.016597


#### Results (rounded)

* **$$ \mathrm{EAD}_{\max} \approx \text{56275440.50} \text{USD} $$**

* **$$ \mathrm{LGD}_{\max} \approx \text{22510176.20} \text{USD} $$**

* **Survival to start of Year 4 $\approx 0.948413 $**

* **Marginal PD in Year 4 $\approx 1.659700\% $**

#### Interpretation 

**(a) Maximum expected exposure (bullet loan)**
$$
\mathrm{EAD}_{\max} = \text{Principal} \times (1+\text{YTM})^{T} = 50\,\text{m} \times (1.03)^4 \approx \text{\$56.275440}\,\text{m}
$$

**(b) Maximum loss given default**
$$
\mathrm{LGD}_{\max} = (1-\text{Recovery}) \times \mathrm{EAD}_{\max} = 0.40 \times 56.275440\,\text{m} \approx \text{\$22.510176}\,\text{m}
$$

**(c) Anticipated default rate in Year 4 (marginal PD)**
$$
\Pr[\text{default in Year 4}] = \text{POD} \times (1-\text{POD})^3 = 0.0175 \times (1-0.0175)^3 \approx 0.016597\ (\text{i.e., }1.659700\%)
$$

---
---

#### Question 2 

In [4]:
# CDS pricing using standard-coupon convention with Effective Duration (RPV01 proxy)
# Assumptions:
# - Univox is HY, so the standard running coupon C = 500 bps
# - EffDurCDS ~ RPV01 (in years) at the given time
# - Upfront ≈ (C - s) * EffDur * Notional   (seller perspective; buyer is opposite sign)
# - Ignore accrual/fees and curve shape (one-factor approximation)

notional = 100_000_000.0  # USD
s0 = 0.0375               # 375 bps at inception
C  = 0.0500               # 500 bps standard coupon for HY
EffDur_0 = 4.8            # at inception

# (a) Inception upfront (seller perspective; + means seller receives from buyer)
upfront_seller = (C - s0) * EffDur_0 * notional
upfront_buyer  = -upfront_seller

# (b) 1y later: spread widens to 575 bps, EffDur=3.75; MTM from seller perspective
s1 = 0.0575
EffDur_1 = 3.75
mtm_seller_t1 = (C - s1) * EffDur_1 * notional

# (c) 2y later: credit event; recovery 0.84 ⇒ seller pays (1 - R) * notional
R = 0.84
settlement = (1 - R) * notional

print("=== (a) Inception ===")
print(f"Seller upfront (USD): {upfront_seller:,.2f}")
print(f"Buyer upfront  (USD): {upfront_buyer:,.2f}")

print("\n=== (b) At t=1y (seller perspective) ===")
print(f"CDS MTM (USD): {mtm_seller_t1:,.2f}")

print("\n=== (c) Credit event at t=2y ===")
print(f"Settlement amount (seller pays, USD): {settlement:,.2f}")


=== (a) Inception ===
Seller upfront (USD): 6,000,000.00
Buyer upfront  (USD): -6,000,000.00

=== (b) At t=1y (seller perspective) ===
CDS MTM (USD): -2,812,500.00

=== (c) Credit event at t=2y ===
Settlement amount (seller pays, USD): 16,000,000.00


#### **Interpretation** 

High-yield (HY) CDS are quoted with a standard running coupon of $C=500$ bps; any gap between the market spread $s$ and $C$ is settled upfront using the risky PV01. Here we use $\text{EffDurCDS}$ as the PV01 proxy:
$$
\text{Upfront} \approx (C - s)\times \text{EffDur}\times N
$$
where $N$ is notional and signs are from the seller’s perspective (positive means seller receives).

#### (a) Who pays upfront and price

At inception $s_0=375$ bps, $C=500$ bps, $\text{EffDur}=4.8$, $N=$100\text{m}$:
$$
\text{Upfront}_{\text{seller}}
= (0.050 - 0.0375)\times 4.8 \times 100{,}000{,}000
= \boxed{6{,}000{,}000}
$$
Because $s_0<C$, the running coupon is too high versus fair $s_0$, so the seller’s position is worth more than par on a running basis and the buyer receives a positive upfront (equal and opposite to the seller).

#### (b) Value after 1 year (seller’s perspective)

With $s_1=575$ bps, $\text{EffDur}=3.75$:
$$
\text{MTM}_{\text{seller}}
\approx (C - s_1)\times 3.75 \times 100{,}000{,}000
= (0.050 - 0.0575)\times 3.75 \times 100{,}000{,}000
= \boxed{-2{,}812{,}500}
$$
Spread widened above the 500 bps coupon, so the seller’s position loses value.

#### (c) Credit event at 2 years; recovery $R=0.84$

Standard settlement (ignoring accrued premium here) is:
$$
\text{Payout} = (1 - R)\times N
= (1 - 0.84)\times 100{,}000{,}000
= \boxed{16{,}000{,}000}
$$
Thus the protection seller pays $16$ million to the buyer at default.


---
---
#### Question 3

In [6]:
# parameters
notional_5y = 100_000_000
notional_10y = 100_000_000

spread_5y_bps = 125    # not used directly in calc (instantaneous price impact needs the change)
spread_10y_bps = 175   # not used directly

EffDur_5y = 4.5
EffDur_10y = 8.5

# Spread changes (narrowing => negative change)
delta_s_5y_bps = -25    # bps p.a.
delta_s_10y_bps = -50   # bps p.a.

# Convert bps to decimal
bps_to_decimal = 1e-4
delta_s_5y = delta_s_5y_bps * bps_to_decimal   # -0.0025
delta_s_10y = delta_s_10y_bps * bps_to_decimal # -0.0050

# (a) Instantaneous excess return from spread moves
# Price impact (approx): ΔP/P ≈ -EffSpreadDur * Δs
gain_5y_pct = -EffDur_5y * delta_s_5y
gain_10y_pct = -EffDur_10y * delta_s_10y

gain_5y_usd = gain_5y_pct * notional_5y
gain_10y_usd = gain_10y_pct * notional_10y
gain_total_usd = gain_5y_usd + gain_10y_usd
portfolio_notional = notional_5y + notional_10y
gain_total_pct = gain_total_usd / portfolio_notional

# (b) Expected excess return over 1 year:
# Expected default loss: PD * LGD * Notional
PD = 0.015          # 1.5% per year
recovery = 0.30
LGD = 1 - recovery  # 70%

expected_loss_usd = PD * LGD * portfolio_notional

# Expected excess return ≈ spread move price gain (from a) - expected loss
expected_excess_return_usd = gain_total_usd - expected_loss_usd
expected_excess_return_pct = expected_excess_return_usd / portfolio_notional

results = {
    "(a) 5y gain %": gain_5y_pct,
    "(a) 10y gain %": gain_10y_pct,
    "(a) 5y gain USD": gain_5y_usd,
    "(a) 10y gain USD": gain_10y_usd,
    "(a) Total gain USD": gain_total_usd,
    "(a) Total gain % of $200m": gain_total_pct,
    "(b) Expected default loss USD": expected_loss_usd,
    "(b) Expected excess return USD": expected_excess_return_usd,
    "(b) Expected excess return % of $200m": expected_excess_return_pct
}
for k,v in results.items():
    print(f"{k}: {v:,.6f}")

(a) 5y gain %: 0.011250
(a) 10y gain %: 0.042500
(a) 5y gain USD: 1,125,000.000000
(a) 10y gain USD: 4,250,000.000000
(a) Total gain USD: 5,375,000.000000
(a) Total gain % of $200m: 0.026875
(b) Expected default loss USD: 2,100,000.000000
(b) Expected excess return USD: 3,275,000.000000
(b) Expected excess return % of $200m: 0.016375


#### **Interpretation** 

#### (a) Instantaneous excess return from spread tightening

For price impact due to a small spread change, use the linear approximation:
$$
\frac{\Delta P}{P};\approx; -,D_{\text{spread}};\Delta s
$$
where $D_{\text{spread}}$ is the effective spread duration and $\Delta s$ is the change in spread in decimal.

* Five-year: $D_{\text{spr}}=4.5$, $\Delta s=-25\ \text{bps}=-0.0025$
  $$
  \frac{\Delta P}{P} = -4.5\times(-0.0025)=0.01125;\Rightarrow;1.125%
  $$
  On $100m: gain $=$1.125\ \text{m}$

* Ten-year: $D_{\text{spr}}=8.5$, $\Delta s=-50\ \text{bps}=-0.0050$
  $$
  \frac{\Delta P}{P} = -8.5\times(-0.0050)=0.0425;\Rightarrow;4.25%
  $$
  On $100m: gain $=$4.25\ \text{m}$

Total instantaneous excess return:
$$
1.125\ \text{m} + 4.25\ \text{m}=\boxed{5.375\ \text{m}}
\quad\text{(i.e., }2.6875%\text{ of $200m)}
$$

#### (b) Expected excess return over one year (include default loss)

Expected default loss with annual default probability $p=1.5% $ and recovery $R=30% $ (thus $LGD=1-R=70% $):
$$
\mathbb{E}[\text{Loss}] = p \times LGD \times \text{Notional}
= 0.015 \times 0.70 \times 200\ \text{m}
= \boxed{2.10\ \text{m}}
$$
Expected excess return $\approx$ spread-move price gain $-$ expected default loss:
$$
5.375\,\text{m} - 2.10\,\text{m}
= \boxed{3.275\,\text{m}}
\quad\text{(i.e., 1.6375\% of \$200m)}
$$

---
---
#### Question 4

In [ ]:
# Compute 5y notional that matches the DV01 (money duration) of a 10y $10mm HY CDX

D5 = 4.65   # 5-year EffSpreadDur_CDS
D10 = 8.75  # 10-year EffSpreadDur_CDS
N10 = 10_000_000  # $10mm notional on the 10y leg

def dv01(notional: float, eff_dur: float) -> float:
    """DV01 to spread (per 1 bp) = notional * EffSpreadDur * 1e-4"""
    return notional * eff_dur * 1e-4

def match_notional(target_dv01: float, eff_dur: float) -> float:
    """Solve for notional needed to match a target DV01 given eff_dur"""
    return target_dv01 / (eff_dur * 1e-4)

# DV01 of the 10y leg
dv01_10y = dv01(N10, D10)

# 5y notional that matches the 10y DV01
N5 = match_notional(dv01_10y, D5)

print(f"DV01 (10y, per 1bp): ${dv01_10y:,.2f}")
print(f"Required 5y notional: ${N5:,.2f}")
print(f"Check DV01 (5y, per 1bp): ${dv01(N5, D5):,.2f}")


DV01 (10y, per 1bp): $8,750.00
Required 5y notional: $18,817,204.30
Check DV01 (5y, per 1bp): $8,750.00


#### **Answer & Interpretation**
#### a) Expected slope change and positioning

**Credit-curve slope (HY)** is commonly measured as $ \text{slope} = s_{10\text{y}} - s_{5\text{y}} $.
From the table today: $s_{5\text{y}}=175$ bps, $s_{10\text{y}}=350$ bps $\Rightarrow$ current slope $=+175$ bps (steep **upward**).

When the economy transitions from **peak growth $\to$ contraction**, near-term default risk jumps and liquidity deteriorates. Front-end HY spreads typically **widen more than** long-end spreads. Hence the HY credit curve tends to **flatten or even invert** (slope **falls** toward 0 or negative).

**How to position with the two indices**

* **Directional recession view (widening):** **Buy protection** on either tenor (5y or 10y) to benefit from spread widening. The 10y has larger $ \text{EffSpreadDur}_{\text{CDS}} $ ($8.75$ vs $4.65$), so for the same notional it is more sensitive to spread moves.

* **Curve-flattening/inversion view (target the slope):**
  Construct a DV01-neutral **5s10s flattener**:

  * **Buy 5y protection** (long spread exposure at the front end).
  * **Sell 10y protection** in a size that matches money duration (so you are long front-end spread and short long-end spread, isolating the slope). Profit if the 5y widens **relative** to the 10y (i.e., the curve flattens/inverts). Reverse the signs for a steepener view.

#### b) 5-year notional that matches the DV01 (money duration) of a 10-year, $10{,}000{,}000$ CDX HY

Money-duration (DV01 to spread) for CDS using the given effective spread duration is
$$
\text{DV01} ;=; N \times D_{\text{eff}} \times 10^{-4},
$$
where $N$ is notional and $D_{\text{eff}}$ is $\text{EffSpreadDur}_{\text{CDS}}$.

Set 5y DV01 equal to 10y DV01:
$$
N_{5},D_{5},10^{-4} ;=; (10{,}000{,}000),D_{10},10^{-4}
;\Rightarrow;
N_{5} ;=; 10{,}000{,}000 \times \frac{D_{10}}{D_{5}} .
$$

Plug in $D_{10}=8.75$, $D_{5}=4.65$:
$$
N_{5}
= 10{,}000{,}000 \times \frac{8.75}{4.65}
= 10{,}000{,}000 \times 1.88172
\approx \boxed{18{,}817{,}000}.
$$

**Interpretation:** A **$18.82$MM** 5-year CDX HY position has the same dollar value of a 1-bp spread move (same money duration) as a **$10$MM** 10-year CDX HY position. This ratio is the hedge weight for building a DV01-neutral 5s10s curve trade.


---
---
#### Question 5

In [ ]:
face = 250_000_000.0      # Face value (USD)
coupon_rate = 0.02        # Annual coupon, as a decimal
y0 = 0.0225               # Yield to maturity at inception
repo_rate = 0.02          # Repo rate (annual, money-market)
initial_margin_pct = 1.03 # 103% initial margin (overcollateralization)
n_years = 10              # Years to maturity at inception

# Margin check on day 3 after a yield shock:
days_total = 10           # Repo tenor (days)
days_to_check = 3         # Day to run margin call
y_after_shock = 0.0275    # New YTM after +50 bps move
day_count = 360           # ACT/360 for repo accrual

# Helper metrics
def bond_price_annual(face: float, coupon_rate: float, ytm: float, n_years: int) -> float:
    """
    Price of an annual-coupon bond (clean, no accrued at t0).
    """
    C = face * coupon_rate
    if ytm == 0:
        return C * n_years + face
    annuity = (1 - (1 + ytm) ** (-n_years)) / ytm
    return C * annuity + face / ((1 + ytm) ** n_years)

def repo_growth_factor(rate: float, days: int, basis: int = 360) -> float:
    """
    Simple money-market accrual over 'days' on an ACT/basis convention.
    """
    return 1.0 + rate * days / basis

# -------- (a) Inception values --------
# Bond market value used as collateral
MV0 = bond_price_annual(face, coupon_rate, y0, n_years)

# Cash lent (purchase price) given 103% initial margin: MV0 = 1.03 * Cash
purchase_price = MV0 / initial_margin_pct

# Initial margin (currency) = Collateral - Cash
initial_margin_amount = MV0 - purchase_price

# Repurchase price after 10 days at repo rate
repurchase_price = purchase_price * repo_growth_factor(repo_rate, days_total, day_count)

# -------- (b) Day-3 margin call after +50 bps YTM move --------
# Reprice collateral with new yield (still 10y to maturity, per problem statement)
MV_day3 = bond_price_annual(face, coupon_rate, y_after_shock, n_years)

# Loan balance with repo accrual to day 3
loan_balance_day3 = purchase_price * repo_growth_factor(repo_rate, days_to_check, day_count)

# Required collateral to maintain 103% margin
required_collateral_day3 = initial_margin_pct * loan_balance_day3

# Margin call amount (positive => security seller must post to security buyer)
margin_call_amount = max(0.0, required_collateral_day3 - MV_day3)
direction = ("Security seller posts to security buyer"
             if margin_call_amount > 0 else
             "Security buyer posts to security seller (or no call if exactly zero)")

# -------- Print results --------
print("=== (a) Inception ===")
print(f"Bond market value (collateral) : ${MV0:,.2f}")
print(f"Purchase price (cash lent)     : ${purchase_price:,.2f}")
print(f"Initial margin (currency)      : ${initial_margin_amount:,.2f}")
print(f"Repurchase price (day 10)      : ${repurchase_price:,.2f}")

print("\n=== (b) Day-3 after +50 bps YTM ===")
print(f"Collateral MV (day 3)          : ${MV_day3:,.2f}")
print(f"Loan balance (day 3)           : ${loan_balance_day3:,.2f}")
print(f"Required collateral (103%)     : ${required_collateral_day3:,.2f}")
print(f"Margin call amount             : ${margin_call_amount:,.2f}")
print(f"Direction                      : {direction}")


=== (a) Inception ===
Bond market value (collateral) : $244,458,614.78
Purchase price (cash lent)     : $237,338,460.95
Initial margin (currency)      : $7,120,153.83
Repurchase price (day 10)      : $237,470,315.65

=== (b) Day-3 after +50 bps YTM ===
Collateral MV (day 3)          : $233,799,857.19
Loan balance (day 3)           : $237,378,017.36
Required collateral (103%)     : $244,499,357.88
Margin call amount             : $10,699,500.69
Direction                      : Security seller posts to security buyer


#### a) Purchase price, initial margin (currency), and repurchase price

**Bond pricing (annual coupons, no accrual at inception):**
Price $=$ PV(coupons) $+$ PV(face)
$$
P_0
=\sum_{t=1}^{10}\frac{0.02\times 250{,}000{,}000}{(1+0.0225)^t}
+\frac{250{,}000{,}000}{(1+0.0225)^{10}}
=244{,}458{,}614.78
$$

**Initial margin 103%** means **collateral value $=1.03\times$ cash**.
So purchase price (cash lent) and initial margin are:
$$
\text{Cash (purchase price)}=\frac{P_0}{1.03}
=\boxed{237{,}338{,}460.95},
\qquad
\text{Initial margin}=P_0-\text{Cash}
=\boxed{7{,}120{,}153.83}.
$$

**Repurchase price** after 10 days at repo rate $2%$ on ACT/360:
$
\displaystyle
\text{Repurchase}=
\text{Cash}\times\Bigl(1+0.02\times\frac{10}{360}\Bigr)
=\boxed{237{,}470{,}315.65}.
$

#### b) Day-3 margin call after a $+50$ bps yield shock (bond still 10y)

Reprice collateral at new YTM $2.75%$:
$$
P_{3}=233{,}799{,}857.19.
$$

Outstanding loan with repo accrual to day 3:
$$
\text{Loan}_{3}=
\text{Cash}\times\Bigl(1+0.02\times\frac{3}{360}\Bigr)
=237{,}378{,}017.36.
$$

To **maintain 103%** over-collateralization, required collateral is
$$
\text{ReqColl}*{3}=1.03\times \text{Loan}*{3}
=244{,}499{,}357.88.
$$

Current collateral is lower than required (price fell as yields rose), so the **margin call** is
$$
\text{Call}=\text{ReqColl}*{3}-P*{3}
=\boxed{10{,}699{,}500.69}.
$$


---
---